# Knapsack Problem

This is an explaination of how to create Genetica algorithm in DEAP to solve the **knapsack problem**. Included are explainations of what each class/fucntion is doing.

### Create a Fitness class and Individual class
The `creator.create` function is for creating new classes. Once created the class can bea accessed using `creator.<class_name>`

In [23]:
from deap import creator
from deap import base
from deap import tools
import numpy as np
import random

# Creates a Fitness class, with a weights attirbute.
#    1 means a metric that needs to be maximized = the value
creator.create("FitnessMax", base.Fitness, weights=(1.0,))

# Creates a class Individual, that is based on a numpy array as that is the
# class used for the weights passed to tensorflow.
#   It also has an attribute that is a Fitness, when setting the attribute
#   it automatically calls the __init__() in Fitness initializing the
#   weight (1)
creator.create("Individual", list, fitness=creator.FitnessMax)


# IGNORE
### Gererate the items
Create a set of random Items to be used to add to the knapsack.
The items will be entered into a dictionary indexed by integers.

# IGNORE
# convert to code if needed

# Create the item dictionary: item name is an integer, and value is 
# a (weight, value) 2-uple.
items = {}
NUM_ITEMS = 10
# Create random items and store them in the items' dictionary.
for i in range(NUM_ITEMS):
    items[i] = (random.randint(1, 10), random.uniform(0, 100))

import pprint
pprint.pprint(items)

### Create evolutionary algorithms
The `base.Toolbox` class is used create evolutionary operators/functions.

`tools.initRepeat` takes a container class and applies the following function, a number of times.


Looking at the RNN implementation:
- num_inputs = 3  # Number of Values output by the simulation
- num_outputs = 1 # OpenAI env takes a single controller input, thus the controller network must output only one.
- num_nodes = num_inputs+1  # Number of nodes in the network (must be >=nump_inputs)

And the number of weights is equal to:
<break>`(num_nodes+num_outputs)*num_nodes`

In [34]:
toolbox = base.Toolbox()

# Create a function 'attr_item' to return the 'ID' of one item
# num_nodes = num_inputs+1 = 20
NUM_NODES = 20

# create a function 'individual'. It takes an individual and instantiates 
#   it with a numpy array of size, NUM_NODES and type float32
toolbox.register("attr_floats", lambda n: np.random.rand(n).astype(np.float32)*0.1, NUM_NODES)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.attr_floats)

# create a function 'population' that generates a list of individuals, x long.
#   NB: As repeat init takes 3 parameters, and as the first 2 have been given.
#       only one is needed.
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

The last thing to do is to define an Evaluation/Fitness function.

In [61]:
def evaluate(individual,MAX_REWARD = 0):
    value = 0.0
    
    for item in individual:
        value+=item             # Ensure overweighted bags are dominated
    return sum(individual), # val

Then applying crosover/ mating.

In [62]:
def cross(ind1, ind2):
    """Apply a crossover operation on input sets. The first child is the
    intersection of the two sets, the second child is the difference of the
    two sets.
    
    Is ind1 the winner or ind2???
    """
    for x in range(len(ind1)):
        if random()<0.5:
            ind2[x] = ind1[x]# Used in order to keep type
                  # Symmetric Difference (inplace)
    return ind1, ind2

Add or remove a random item.

In [63]:
def mutate(individual):
    """Mutation that pops or add an element"""
    #     if random.random() < (1/len(individual)):

    # Select one gene
    index = random.randint(0,len(individual)-1)
    individual[index] *= (np.random.randn()*0.0025)
    return individual

Since we are using a multi-objective problem we have selected the SPEA-II selection scheme... wtf.

In [77]:
toolbox.register("evaluate", evaluate)
toolbox.register("mate", cross)
toolbox.register("mutate", mutate)
toolbox.register("select", tools.selTournament, tournSize=2)

In [87]:
import numpy
from deap import algorithms

def main():
    pop = toolbox.population(n=50)
    CXPB, MUTPB, NGEN = 0.5, 0.2, 40
    
    # Evaluate the entire population
    fitnesses = map(toolbox.evaluate, pop)
    print(pop)
    for ind, fit in zip(pop, fitnesses):
        print()
        break
        ind.fitness.values = fit

    for g in range(NGEN):
        # Select the next generation individuals
        offspring = toolbox.select(pop, len(pop))
        # Clone the selected individuals
        offspring = map(toolbox.clone, offspring)

        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CXPB:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        # The population is entirely replaced by the offspring
        pop[:] = offspring

    return pop


# def main():
#     pop = toolbox.population(n=300)
#     hof = tools.HallOfFame(1)
#     stats = tools.Statistics(lambda ind: ind.fitness.values)
#     stats.register("avg", numpy.mean)
#     stats.register("std", numpy.std)
#     stats.register("min", numpy.min)
#     stats.register("max", numpy.max)
    
#     pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=40, 
#                                    stats=stats, halloffame=hof, verbose=True)
    
    return pop, stats, hof

In [88]:
pop, stats, hof = main()

[[0.094320983, 0.060847271, 0.091328911, 0.076275706, 0.0020662632, 0.054132927, 0.031428955, 0.097585991, 0.0058827861, 0.052391496, 0.016461795, 0.036221426, 0.03109988, 0.051949609, 0.022814957, 0.086360238, 0.0063173412, 0.071859084, 0.06676998, 0.064989097], [0.071873724, 0.043176029, 0.042063855, 0.054440357, 0.045165155, 0.035842363, 0.054598916, 0.083740853, 0.020161796, 0.05485893, 0.029422704, 0.08655329, 0.018990554, 0.089087732, 0.036557209, 0.026101682, 0.011408357, 0.069620244, 0.088161819, 0.019087235], [0.084538735, 0.072524831, 0.073958568, 0.023848614, 0.057635363, 0.011350655, 0.020907894, 0.013557129, 0.048789654, 0.057688415, 0.0041830651, 0.030123292, 0.016003838, 0.079330556, 0.087103382, 0.078468464, 0.087213486, 0.046929035, 0.094680324, 0.044002239], [0.04600824, 0.013672261, 0.068236396, 0.0046358886, 0.059943642, 0.0742504, 0.052952792, 0.032373764, 0.06924931, 0.030114977, 0.069884419, 0.075275503, 0.0383876, 0.070411146, 0.070495263, 0.086491607, 0.0177327

TypeError: selTournament() got an unexpected keyword argument 'tournSize'

# NEXT TIME: MICROBIAL GA FOR REAL VALUES.